In [14]:
!poetry update

Updating dependencies
Resolving dependencies... (9.8s)Resolving dependencies... (2.6s)

No dependencies to install or update


# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    google_colab = importlib.import_module('google.colab')
    google_colab.drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [13]:
target_period_hours = 2 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1600. * 2./100.)

target expiry: 2024-01-28 08:00:00+00:00 (38 hours left)
current price: 2269.02
-------------------------------
trading put 2250 and call 2300
hit ratio: 56%
cost: 0.008 / value: 0.024, benefit/cost = 3.0x
($) cost: 18.15 / value: 54.07, average gain = 35.92
-------------------------------
trading put 2225 and call 2325
hit ratio: 46%
cost: 0.004 / value: 0.017, benefit/cost = 4.5x
($) cost: 8.62 / value: 39.06, average gain = 30.44
-------------------------------
trading put 2200 and call 2350
hit ratio: 35%
cost: 0.002 / value: 0.012, benefit/cost = 5.6x
($) cost: 4.99 / value: 28.17, average gain = 23.18
-------------------------------
trading put 2175 and call 2375
hit ratio: 26%
cost: 0.001 / value: 0.009, benefit/cost = 6.9x
($) cost: 2.95 / value: 20.21, average gain = 17.26


[{'put': 2225.0, 'call': 2325.0, 'quantity': 2},
 {'put': 2200.0, 'call': 2350.0, 'quantity': 3}]

In [3]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2175.0,104.703768,NaN,0.0505,10.726132,0.004733,0.0007
2200.0,83.855854,NaN,0.0350,14.878218,0.006565,0.0012
2225.0,64.641264,NaN,0.0250,20.663628,0.009118,0.0024
2250.0,47.826394,NaN,0.0125,28.848758,0.012730,0.0050
2275.0,34.671766,0.015300,0.0065,40.694130,NaN,0.0100
2300.0,25.300643,0.011165,0.0031,56.323007,NaN,NaN
2325.0,18.392156,0.008116,0.0017,74.414520,NaN,NaN
2350.0,13.254893,0.005849,0.0010,94.277257,NaN,0.0480
2375.0,9.433695,0.004163,0.0006,115.456059,NaN,0.0600
